## EDA - Pornography dataset

### Intro and goals

Pornography has been a very controversial subject on social life and it is as commom as it is a taboo. Recently, neuroscientits are warning of the harm of porn videos, how it affects relationships and social interactions and its scalable behavior (one day a simple sex video may be not enough).

As a curious person, I tried to find data about this discovers and found this dataset.

This is [dataset](https://mazieres.gitlab.io/sexualitics/index.html) that I first found at FiveThirtyEight. It consists of nearly 800k entries of metadatas from a pornography site from its creation until 2013 and it was used on [this article](https://hal.archives-ouvertes.fr/hal-00937745v2/document).

So my goal on this notebook is to answer some simple questions! 

Is there any correlation between runtime and views? Or comments? And what about votes?

What is the most used tags? Is there any words that could possibly make the video be seen more times or more voted than the others?

Weeeell, let's see...

### First things

I'll be using SweetViz as AutoEDA tool and complementing with Altair/Seaborn and Pandas when needed.

In [4]:
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns
import sweetviz as sv
import re

%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [5]:
df = pd.read_csv("xhamster.csv")

### Analyzing

In [6]:
# Creating and exporting de report as html

report = sv.analyze(df).show_html("Study.html")

                                             |          | [  0%]   00:00 -> (? left)

Report Study.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


We have few missing values and no duplicates, that's good.

There's videos with 8,3M views or more than 17k votes or even almost 1.000 comments!! Surprisingly or not, that's a whole world of engagement... I'll be exploring this later on.

From SweetViz report, it's possible to see that the runtime has no correlation with other features, but, on the other hand, votes and comments are correlated with each other and mostly with views. 

AutoEDA is quick, indeed.

After this overview, I'm going to clean the dataset and remove the NaN rows (less than 8%).

In [7]:
# Droping columns that will not be used
df = df.drop(columns=["uploader", "upload_date", "description", "title"]).dropna()
df.head()

,id,channels,nb_views,nb_votes,nb_comments,runtime
0,378466,"['BBW', 'Black and Ebony', 'Interracial']",17262.0,65.0,11.0,120.0
2,287146,"['Babes', 'Teens', 'Webcams']",6060.0,11.0,3.0,163.0
3,378462,['Men'],12742.0,87.0,15.0,1980.0
4,1583073,['Gays'],32879.0,75.0,6.0,1318.0
5,378468,"['Amateur', 'Voyeur']",33778.0,62.0,8.0,625.0


In [8]:
# Here, we start to create the future tag's columns
tag_df = df["channels"].str.split(",", expand=True)
tag_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,['BBW','Black and Ebony','Interracial'],None,None,None,None,None,None,None,None,None
2,['Babes','Teens','Webcams'],None,None,None,None,None,None,None,None,None
3,['Men'],None,None,None,None,None,None,None,None,None,None,None
4,['Gays'],None,None,None,None,None,None,None,None,None,None,None
5,['Amateur','Voyeur'],None,None,None,None,None,None,None,None,None,None


In [9]:
tag_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 726301 entries, 0 to 786120
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       726301 non-null  object
 1   1       543340 non-null  object
 2   2       421365 non-null  object
 3   3       38293 non-null   object
 4   4       16834 non-null   object
 5   5       2624 non-null    object
 6   6       1442 non-null    object
 7   7       831 non-null     object
 8   8       416 non-null     object
 9   9       118 non-null     object
 10  10      41 non-null      object
 11  11      10 non-null      object
dtypes: object(12)
memory usage: 72.0+ MB


In [10]:
# Removing last columns, since it makes hard to process and does not aggregate that much information
tag_df = tag_df.drop(tag_df.iloc[:, 5:12], axis=1)
tag_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 726301 entries, 0 to 786120
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       726301 non-null  object
 1   1       543340 non-null  object
 2   2       421365 non-null  object
 3   3       38293 non-null   object
 4   4       16834 non-null   object
dtypes: object(5)
memory usage: 33.2+ MB


In [11]:
# Since it's confusing to work with columns names like numbers, I decided to rename them here

tag_df.rename(
    columns={
        tag_df.columns[0]: "Tag1",
        tag_df.columns[1]: "Tag2",
        tag_df.columns[2]: "Tag3",
        tag_df.columns[3]: "Tag4",
        tag_df.columns[4]: "Tag5",
    },
    inplace=True,
)
tag_df.head()

,Tag1,Tag2,Tag3,Tag4,Tag5
0,['BBW','Black and Ebony','Interracial'],None,None
2,['Babes','Teens','Webcams'],None,None
3,['Men'],None,None,None,None
4,['Gays'],None,None,None,None
5,['Amateur','Voyeur'],None,None,None


In [12]:
# Replacing the [] and ''

i = 0
while i <= 4:
    tag_df = pd.concat(
        [
            tag_df,
            tag_df.iloc[:, i]
            .str.replace(r"[\'\[\]]", "", regex=True)
            .rename("tag_%d" % i),
        ],
        axis=1,
    )
    i = i + 1

cols = list(tag_df.columns)
tag_df[cols] = tag_df[cols].apply(lambda x: x.str.strip())

In [13]:
tag_df = tag_df.drop(columns=["Tag1", "Tag2", "Tag3", "Tag4", "Tag5"])

In [14]:
tag_df.head()

,tag_0,tag_1,tag_2,tag_3,tag_4
0,BBW,Black and Ebony,Interracial,None,None
2,Babes,Teens,Webcams,None,None
3,Men,None,None,None,None
4,Gays,None,None,None,None
5,Amateur,Voyeur,None,None,None


In [15]:
# Creating a Tag dataset with the frequency of each word

tag = (
    tag_df["tag_0"]
    .value_counts()
    .reset_index()
    .merge(tag_df["tag_1"].value_counts().reset_index(), on="index", how="outer")
    .merge(tag_df["tag_2"].value_counts().reset_index(), on="index", how="outer")
    .merge(tag_df["tag_3"].value_counts().reset_index(), on="index", how="outer")
    .merge(tag_df["tag_4"].value_counts().reset_index(), on="index", how="outer")
)
tag.fillna(0, inplace=True)

tag["total"] = tag["tag_0"] + tag["tag_1"] + tag["tag_2"] + tag["tag_3"] + tag["tag_4"]
tag.replace("", np.nan, inplace=True)
tag = tag.dropna()

tag.sort_values("total", ascending=False, inplace=True)
tag.rename(columns={"index": "Tag"}, inplace=True)
tag = tag.reset_index().drop(columns="index")
tag.head()

,Tag,tag_0,tag_1,tag_2,tag_3,tag_4,total
0,Amateur,206914,224.0,191.0,24.0,65.0,207418.0
1,Men,92505,18273.0,3880.0,0.0,0.0,114658.0
2,Teens,4061,17660.0,46970.0,1837.0,1735.0,72263.0
3,Hardcore,11713,32889.0,24917.0,1291.0,328.0,71138.0
4,Anal,48357,12920.0,26.0,9.0,18.0,61330.0


In [16]:
# Plotting the firsts and lasts ones

upper = (
    alt.Chart(tag[:10])
    .mark_bar()
    .encode(alt.X("Tag", sort="-y"), y="total",)
    .properties(width=1200, height=400)
)

lower = (
    alt.Chart(tag[-10:])
    .mark_bar()
    .encode(alt.X("Tag", sort="-y"), y="total",)
    .properties(width=1200, height=400)
)


upper & lower

alt.VConcatChart(...)

So, the most used tag is by far "Amateur" followed by "Men" and a little further away we have "Teens", "Hardcore", "Blowjobs" and "Anal". On the other hand, the least used are "Midget", "Gothics", "Babysitters" and "Swedish"

In [17]:
tag_df.value_counts().reset_index()

,tag_0,tag_1,tag_2,tag_3,tag_4,0
0,Amateur,Blowjobs,Cumshots,Facials,Handjobs,131
1,Blowjobs,Cumshots,Facials,Handjobs,Vintage,66
2,Amateur,Hardcore,Public Nudity,Teens,Upskirts,46
3,Babes,Brunettes,Celebrities,Softcore,Tits,41
4,Amateur,Lesbians,Matures,MILFs,Old+Young,41
...,...,...,...,...,...,...
11877,Amateur,Latin,POV,Teens,Voyeur,1
11878,Amateur,Latin,Public Nudity,Softcore,Teens,1
11879,Amateur,Latin,Sex Toys,Squirting,Webcams,1
11880,Amateur,Latin,Showers,Tits,Voyeur,1


And what about combinations of tags? Well, as its shown above, using "Amateur", "Blowjobs", "Cumshots", "Facials" and "Handjobs" is twice as common than the second set of tags. 

Could this words or set of words being used bring more views or comments? Let's find out...

In [18]:
# First, I'm going to join the dfs
df = pd.concat([df, tag_df], axis=1)

In [19]:
# Rearranging the datset, droping `channels` column and removing empty rows

df = df[
    [
        "id",
        "tag_0",
        "tag_1",
        "tag_2",
        "tag_3",
        "tag_4",
        "nb_views",
        "nb_votes",
        "nb_comments",
        "runtime",
    ]
]
df.replace("", np.nan, inplace=True)
df.dropna(inplace=True)
df.head()

,id,tag_0,tag_1,tag_2,tag_3,tag_4,nb_views,nb_votes,nb_comments,runtime
32,1698371,Asian,Japanese,Lesbians,Sex Toys,Teens,36934.0,53.0,4.0,365.0
52,1641085,Big Boobs,Blowjobs,Cumshots,Pornstars,Tits,8541.0,10.0,3.0,661.0
119,1126828,Asian,Blowjobs,Cream Pie,Interracial,Thai,148039.0,270.0,22.0,657.0
155,1717892,Amateur,Fingering,Matures,Masturbation,MILFs,28963.0,45.0,5.0,80.0
257,105630,Anal,Hardcore,Matures,MILFs,French,1151898.0,1608.0,40.0,439.0


In [20]:
# Before starting, I'll be dealing with some pretty ugly outliers from 'runtime' variable

cols = df.select_dtypes("number").columns
df_aux = df.loc[:, cols]

iqr = df_aux.quantile(0.75, numeric_only=False) - df_aux.quantile(
    0.25, numeric_only=False
)
lim = np.abs((df_aux - df_aux.median()) / iqr) < 2.22

df.loc[:, cols] = df_aux.where(lim, np.nan)
df["nb_views"].fillna(df["nb_views"].median(), inplace=True)
df["nb_votes"].fillna(df["nb_votes"].median(), inplace=True)
df["nb_comments"].fillna(df["nb_comments"].median(), inplace=True)
df["runtime"].fillna(df["runtime"].median(), inplace=True)
# Creating a df for tags and the sum of each numeric feature

# Dataset for views
tag_views = pd.concat(
    [
        df.groupby("tag_0")
        .sum()
        .reset_index()[["tag_0", "nb_views"]]
        .rename(columns={"tag_0": "Tag"}),
        df.groupby("tag_1")
        .sum()
        .reset_index()[["tag_1", "nb_views"]]
        .rename(columns={"tag_1": "Tag"}),
        df.groupby("tag_2")
        .sum()
        .reset_index()[["tag_2", "nb_views"]]
        .rename(columns={"tag_2": "Tag"}),
        df.groupby("tag_3")
        .sum()
        .reset_index()[["tag_3", "nb_views"]]
        .rename(columns={"tag_3": "Tag"}),
        df.groupby("tag_4")
        .sum()
        .reset_index()[["tag_4", "nb_views"]]
        .rename(columns={"tag_4": "Tag"}),
    ]
).reset_index(drop=True)

tag_views = (
    tag_views.groupby("Tag")
    .sum()
    .sort_values("nb_views", ascending=False)
    .reset_index()
)


# Dataset for votes
tag_votes = pd.concat(
    [
        df.groupby("tag_0")
        .sum()
        .reset_index()[["tag_0", "nb_votes"]]
        .rename(columns={"tag_0": "Tag"}),
        df.groupby("tag_1")
        .sum()
        .reset_index()[["tag_1", "nb_votes"]]
        .rename(columns={"tag_1": "Tag"}),
        df.groupby("tag_2")
        .sum()
        .reset_index()[["tag_2", "nb_votes"]]
        .rename(columns={"tag_2": "Tag"}),
        df.groupby("tag_3")
        .sum()
        .reset_index()[["tag_3", "nb_votes"]]
        .rename(columns={"tag_3": "Tag"}),
        df.groupby("tag_4")
        .sum()
        .reset_index()[["tag_4", "nb_votes"]]
        .rename(columns={"tag_4": "Tag"}),
    ]
).reset_index(drop=True)

tag_votes = (
    tag_votes.groupby("Tag")
    .sum()
    .sort_values("nb_votes", ascending=False)
    .reset_index()
)

# Dataset for comments
tag_comments = pd.concat(
    [
        df.groupby("tag_0")
        .sum()
        .reset_index()[["tag_0", "nb_comments"]]
        .rename(columns={"tag_0": "Tag"}),
        df.groupby("tag_1")
        .sum()
        .reset_index()[["tag_1", "nb_comments"]]
        .rename(columns={"tag_1": "Tag"}),
        df.groupby("tag_2")
        .sum()
        .reset_index()[["tag_2", "nb_comments"]]
        .rename(columns={"tag_2": "Tag"}),
        df.groupby("tag_3")
        .sum()
        .reset_index()[["tag_3", "nb_comments"]]
        .rename(columns={"tag_3": "Tag"}),
        df.groupby("tag_4")
        .sum()
        .reset_index()[["tag_4", "nb_comments"]]
        .rename(columns={"tag_4": "Tag"}),
    ]
).reset_index(drop=True)

tag_comments = (
    tag_comments.groupby("Tag")
    .sum()
    .sort_values("nb_comments", ascending=False)
    .reset_index()
)

# Dataset for runtime
tag_runtime = pd.concat(
    [
        df.groupby("tag_0")
        .sum()
        .reset_index()[["tag_0", "runtime"]]
        .rename(columns={"tag_0": "Tag"}),
        df.groupby("tag_1")
        .sum()
        .reset_index()[["tag_1", "runtime"]]
        .rename(columns={"tag_1": "Tag"}),
        df.groupby("tag_2")
        .sum()
        .reset_index()[["tag_2", "runtime"]]
        .rename(columns={"tag_2": "Tag"}),
        df.groupby("tag_3")
        .sum()
        .reset_index()[["tag_3", "runtime"]]
        .rename(columns={"tag_3": "Tag"}),
        df.groupby("tag_4")
        .sum()
        .reset_index()[["tag_4", "runtime"]]
        .rename(columns={"tag_4": "Tag"}),
    ]
).reset_index(drop=True)

tag_runtime = (
    tag_runtime.groupby("Tag")
    .sum()
    .sort_values("runtime", ascending=False)
    .reset_index()
)

In [21]:
tag_comments

,Tag,nb_comments
0,Amateur,67263.0
1,Blowjobs,38975.0
2,Teens,33458.0
3,Anal,29777.0
4,Big Boobs,29272.0
...,...,...
82,Hentai,315.0
83,Danish,254.0
84,Swedish,228.0
85,Cartoons,125.0


In [22]:
# Creating charts with altair and then concatenating them
r_views = (
    alt.Chart(tag_views[:10])
    .mark_bar()
    .encode(alt.Y("Tag", sort="-x"), x="nb_views",)
    .properties(width=500, height=500)
)

l_views = (
    alt.Chart(tag_views[-10:])
    .mark_bar()
    .encode(alt.Y("Tag", sort="-x"), x="nb_views")
    .properties(width=500, height=500)
)

r_votes = (
    alt.Chart(tag_votes[:10])
    .mark_bar()
    .encode(alt.Y("Tag", sort="-x"), x="nb_votes",)
    .properties(width=500, height=500)
)

l_votes = (
    alt.Chart(tag_votes[-10:])
    .mark_bar()
    .encode(alt.Y("Tag", sort="-x"), x="nb_votes")
    .properties(width=500, height=500)
)

r_comments = (
    alt.Chart(tag_comments[:10])
    .mark_bar()
    .encode(alt.Y("Tag", sort="-x"), x="nb_comments",)
    .properties(width=500, height=500)
)

l_comments = (
    alt.Chart(tag_comments[-10:])
    .mark_bar()
    .encode(alt.Y("Tag", sort="-x"), x="nb_comments")
    .properties(width=500, height=500)
)

r_runtime = (
    alt.Chart(tag_runtime[:10])
    .mark_bar()
    .encode(alt.Y("Tag", sort="-x"), x="runtime",)
    .properties(width=500, height=500)
)

l_runtime = (
    alt.Chart(tag_runtime[-10:])
    .mark_bar()
    .encode(alt.Y("Tag", sort="-x"), x="runtime")
    .properties(width=500, height=500)
)

alt.vconcat(
    alt.hconcat(r_views, l_views, title="Tags x Views"),
    alt.hconcat(r_votes, l_votes, title="Tags x Votes"),
    alt.hconcat(r_comments, l_comments, title="Tags x Comments"),
    alt.hconcat(r_runtime, l_runtime, title="Tags x Runtime"),
).configure_axis(labelFontSize=16).configure_title(anchor="middle", fontSize=18)

alt.VConcatChart(...)

"Amateur" is the tag that has more views, votes and comments by far, although it is only third place on total runtime. It seems that there is a reason for "Amateur" being so used (or maybe its engagement makes uploaders use it, who knows...).

Furthermore, "Amateur" can be considered an "efficient" tag, since it has more engagement from users with only the third largest total runtime.

At the bottom, it's possible to notice some tags that can be associated with very specifics social groups or fetishs like "Midgets", "Cartoons" and "Gothics"... So, that's maybe the reason for its small numbers, it only reachs a small audience.

Now, I'm going to check out for relation between variables.

In [23]:
# Creating a scatter matrix

alt.data_transformers.disable_max_rows()

alt.Chart(df).mark_circle().encode(
    alt.X(alt.repeat("column"), type="quantitative"),
    alt.Y(alt.repeat("row"), type="quantitative"),
    color="Origin:N",
).properties(width=150, height=150).repeat(
    row=["runtime", "nb_views", "nb_votes", "nb_comments"],
    column=["runtime", "nb_views", "nb_votes", "nb_comments"],
)

alt.RepeatChart(...)

Apparently there's no easy noticeable correlation between the variables. Though, it's possible to see a small tendency on views and other features, but runtime, also a correlation between comments and votes.

Now, I'm going to create a new feature, a metric to the video engagement efficency. I'll sum all engagement metrics (votes, coments and views) and divide by how many times that specific tag appeared.

In [24]:
# Creating the dataset

tag_eff = (
    tag.drop(columns=["tag_0", "tag_1", "tag_2", "tag_3", "tag_4"])
    .merge(tag_views, on="Tag")
    .merge(tag_votes, on="Tag")
    .merge(tag_comments, on="Tag")
)

# Calculating the new column

tag_eff["eff"] = round(
    (tag_eff.nb_views + tag_eff.nb_votes + tag_eff.nb_comments) / tag_eff.total
)

tag_eff = tag_eff.sort_values("eff", ascending=False).reset_index(drop=True)

tag_eff.head()

,Tag,total,nb_views,nb_votes,nb_comments,eff
0,Swingers,2981.0,25268382.0,40699.0,4228.0,8492.0
1,Gaping,2760.0,16559753.0,33656.0,3405.0,6013.0
2,Babysitters,614.0,3231854.0,5679.0,452.0,5274.0
3,Tits,19415.0,97577745.0,173642.0,18837.0,5036.0
4,Facials,21721.0,107158124.0,201792.0,21537.0,4944.0


In [25]:
# PLoting the top ones and last ones

upper = (
    alt.Chart(tag_eff[:10])
    .mark_bar()
    .encode(alt.X("Tag", sort="-y"), y="eff",)
    .properties(width=1200, height=500)
)

lower = (
    alt.Chart(tag_eff[-10:])
    .mark_bar()
    .encode(alt.X("Tag", sort="-y"), y="eff")
    .properties(width=1200, height=500)
)

alt.vconcat(upper, lower).configure_axis(labelFontSize=16)

alt.VConcatChart(...)

Surprisingly, we have tags that hasn't appeared before here. "Swingers" and "Gaping" are the first ones and if we look back, they're not the least used. Meanwhile, "Babysitters" is the 3rd, but was little used, that could be the reason to the highlight here.

On the bottom side, there are some known tags that seems to be little used and get little engagement, like "Cartoons" or "Midgets". And another highlight was "Amateur" that in spite of being frequently used appears on 10th least used (maybe 'cause it's indeed very used).

### Conclusion

It was a little hard to handle with the tag's (channel columns). A string with symbols and many different words at first...

In the end, we were able to achieve our initial objectives by identifying relations, the most commonly used words(tags) and even created a new metric for them.

If you know some datasets/articles about pornography please let me know, that's an interesting social aspect which I look forward to learn more.

Feel free to contribute. I'm learner.

That's all for today! ;)